In [265]:
import pandas as pd
import numpy as np

In [266]:
df = pd.read_csv('dirty_cafe_sales.csv')

In [267]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [268]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [269]:
df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_9226047,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [270]:
# Fix columns
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

In [290]:
# Clean transaction_id column
print(df['transaction_id'].nunique())
# Check to see if all transaction IDs follow the pattern "TXN_" followed by digits
valid_id = df["transaction_id"].str.match(r"^TXN_\d+$", na=False)
valid_id.sum()


10000


np.int64(10000)

In [272]:
df['item'].unique()

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'UNKNOWN',
       'Sandwich', nan, 'ERROR', 'Juice', 'Tea'], dtype=object)

In [273]:
# Clean item column
# Replace common invalid entries with NaN
df['item'] = df['item'].replace(['UNKOWN', 'N/A', '', 'ERROR', 'NONE', 'Unknown'], np.nan)
df['item'] = df['item'].str.strip().str.title()
df['item'].value_counts(dropna=False)

item
Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
NaN          625
Unknown      344
Name: count, dtype: int64

In [274]:
df.head()

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [275]:
# CLean quantity column
df['quantity'].unique()

array(['2', '4', '5', '3', '1', 'ERROR', 'UNKNOWN', nan], dtype=object)

In [276]:
df['quantity'].value_counts(dropna=False)
# Convert to numeric and integer
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
df['quantity'] = df['quantity'].astype('Int64')
# Replace NaN with median
df['quantity'] = df['quantity'].replace(np.nan, df['quantity'].median())
df['quantity'].unique()

<IntegerArray>
[2, 4, 5, 3, 1]
Length: 5, dtype: Int64

In [277]:
df['price_per_unit'].unique()

array(['2.0', '3.0', '1.0', '5.0', '4.0', '1.5', nan, 'ERROR', 'UNKNOWN'],
      dtype=object)

In [ ]:
# Clean price_per_unit column
# Convert to numeric
df['price_per_unit'] = pd.to_numeric(df['price_per_unit'], errors='coerce')
# Replace NaN with median
df['price_per_unit'] = df['price_per_unit'].replace(np.nan, df['price_per_unit'].median())

In [279]:
df['total_spent'].unique()

array(['4.0', '12.0', 'ERROR', '10.0', '20.0', '9.0', '16.0', '15.0',
       '25.0', '8.0', '5.0', '3.0', '6.0', nan, 'UNKNOWN', '2.0', '1.0',
       '7.5', '4.5', '1.5'], dtype=object)

In [280]:
# Clean total_spent column
df["total_spent"] = pd.to_numeric(df["total_spent"], errors="coerce")
# Calculate total_spent based on quantity and price_per_unit
df['new_total_spent'] = df['quantity'] * df['price_per_unit']
# Replace total_spent with new_total_spent
df.loc[df['total_spent'] != df['new_total_spent'], 'total_spent'] = df['new_total_spent']
df = df.drop(columns=['new_total_spent'])

In [281]:
check_total = df['quantity'] * df['price_per_unit']
mismatch = df["total_spent"] != check_total
mismatch.sum()  # Should be 0 if all total_spent values are correct

np.int64(0)

In [282]:
df['payment_method'].unique()


array(['Credit Card', 'Cash', 'UNKNOWN', 'Digital Wallet', 'ERROR', nan],
      dtype=object)

In [283]:
# Fix payment_method column
# Replace invalid entries with NaN
df["payment_method"] = df["payment_method"].replace(["UNKNOWN", "ERROR"], np.nan)
# Standardize formatting
df['payment_method'] = df['payment_method'].str.strip().str.title()

In [284]:
df['location'].unique()

array(['Takeaway', 'In-store', 'UNKNOWN', nan, 'ERROR'], dtype=object)

In [285]:
# Fix location column
df["location"] = df["location"].replace(["UNKNOWN", "ERROR"], np.nan)
df['location'] = df['location'].str.strip().str.title()

In [286]:
# Fix transaction_date column
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')
# Create transaction_year column
df['Week_day'] = df['transaction_date'].dt.day_name()
df.sample(10)

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date,Week_day
7170,TXN_2929426,Salad,5,5.0,25.0,Cash,Takeaway,2023-01-29,Sunday
1426,TXN_5121322,Sandwich,3,4.0,12.0,NaN,In-Store,2023-12-17,Sunday
6668,TXN_6376329,Cake,1,3.0,3.0,NaN,Takeaway,2023-03-30,Thursday
5242,TXN_6196052,Smoothie,5,4.0,NaN,Cash,Takeaway,2023-04-07,Friday
2799,TXN_7433758,Cookie,1,1.0,1.0,Cash,NaN,2023-06-09,Friday
7755,TXN_2009782,Salad,5,5.0,25.0,Credit Card,NaN,2023-05-11,Thursday
7027,TXN_1390917,Sandwich,1,4.0,4.0,Credit Card,NaN,2023-01-16,Monday
471,TXN_3414935,Coffee,4,2.0,8.0,NaN,In-Store,2023-08-03,Thursday
5198,TXN_8847610,Juice,3,3.0,9.0,Digital Wallet,Takeaway,2023-12-06,Wednesday
652,TXN_9239779,Salad,1,5.0,5.0,Cash,NaN,2023-01-09,Monday


In [287]:
df.to_csv('clean_cafe_sales.csv', index=False)